# Exploratory Data Cleaning - Griff

## We want to answer the question: What movie genre should our studio invest in to make the most profit.


### In the cells below, IMDB, TMDB, and 'The Numbers' Datasets are imported into Pandas DataFrames for cleaning.  The following datasets are used:
- imdb name basics
- imdb title basics
- imdb title crew
- imdb title principals
- imdb title ratings
- tmdb movies
- tn movie budgets

### Pandas and Numpy are used for cleaning and analysis.  Matplotlib is for visualization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### imdb.name.basics is imported and the birth_year, death_year, primary_profession, and known_for_titles columns are dropped because they will not be a part of this analysis.

In [ ]:
imdb_name_basics = pd.read_csv('./data/imdb.name.basics.csv')
imdb_name_basics.drop(columns = {'birth_year', 
                                 'death_year',
                                 'primary_profession',
                                 'known_for_titles'},inplace=True)
imdb_name_basics.head()

### imdb.title.basics is imported and the column original_title is dropped because only the primary_title will be used.  The variable we will be examining is genres.  Rows with missing values for genres will be dropped because they can't be replaced.  Runtime_minutes might be examined in the future. Missing values for runtime_minutes will be replaced with the median value. Median = 87 minutes, Mean = 86 minutes.

In [ ]:
imdb_title_basics = pd.read_csv('./data/imdb.title.basics.csv')
imdb_title_basics.drop(columns = {'original_title'}, inplace = True)
imdb_title_basics = imdb_title_basics.loc[imdb_title_basics.genres.isna() == False]
imdb_title_basics.runtime_minutes.fillna(imdb_title_basics.runtime_minutes.median(), inplace = True)
# imdb_title_basics.genres = imdb_title_basics.genres.apply(lambda x: x.split(','))
imdb_title_basics.head()

#### From the imdb.title.crew file, the writers column is dropped.  The directors column might be used in the future?

In [ ]:
imdb_title_crew = pd.read_csv('./data/imdb.title.crew.csv')
imdb_title_crew.drop(columns = 'writers', inplace = True)
imdb_title_crew = imdb_title_crew.dropna()
# imdb_title_crew.directors = imdb_title_crew.directors.apply(lambda x: x.split(','))
imdb_title_crew.head()

#### The imdb.title.principals file has role data for movies.  The job and characters columns are almost entirely incomplete and are dropped.  This data might be used later?

In [ ]:
imdb_title_principals = pd.read_csv('./data/imdb.title.principals.csv')
imdb_title_principals.drop(columns = {'job', 'characters'}, inplace = True)
imdb_title_principals.head()

#### imdb.title.ratings has rating data.  We might use this at some point??

In [ ]:
imdb_title_ratings = pd.read_csv('./data/imdb.title.ratings.csv')
imdb_title_ratings = imdb_title_ratings.loc[imdb_title_ratings.numvotes >5]
imdb_title_ratings.head()

#### The tmdb.movies file has info about genre, original lanuage, original title, popularity, release date, and voting.  

In [ ]:
tmdb_movies = pd.read_csv('./data/tmdb.movies.csv')
tmdb_movies.head()
# tmdb_movies = tmdb_movies.drop(columns = {'original_language', 'release_date'   , 'original_title'}).set_index('Unnamed: 0')

### This function removes the dollar sign and commas from a string, and turns it into a number

In [ ]:
def clean_columns(series):
    return series.apply(lambda x: float(x.replace('$', '').replace(',', '')))

### tn.movie_budgets contains financial data that will be used in the comparison of genres.  The redundant ID column is dropped, and clean_columns() function is used to format the columns with money elements.

In [ ]:
tn_movie_budgets = pd.read_csv('./data/tn.movie_budgets.csv')
tn_movie_budgets = tn_movie_budgets.drop(columns={'id'})
tn_movie_budgets.production_budget = clean_columns(tn_movie_budgets.production_budget)
tn_movie_budgets.domestic_gross = clean_columns(tn_movie_budgets.domestic_gross)
tn_movie_budgets.worldwide_gross = clean_columns(tn_movie_budgets.worldwide_gross)
tn_movie_budgets.release_date

### A new column is added, budget_gross_ratio, that is the ratio of worldwide_gross to production_budget.
#### Values <1 indicate the movie grossed less than the budget.

In [ ]:
tn_movie_budgets['budget_gross_ratio'] = tn_movie_budgets.worldwide_gross / tn_movie_budgets.production_budget
tn_movie_budgets = tn_movie_budgets.loc[tn_movie_budgets.domestic_gross > 0]
tn_movie_budgets.head()

# Exploratory Data Analysis

### Below, the heads of each dataset are shown for viewing pleasure, and to help associate the sets with each other.  The get_name function takes a 'nconst' unique identifier and returns the name associated with that id.

In [ ]:
def get_name(name):
    return imdb_name_basics.loc[imdb_name_basics.nconst == name]

In [ ]:
imdb_name_basics.head()

In [ ]:
imdb_title_basics.head()

In [ ]:
imdb_title_ratings.head()

In [ ]:
imdb_title_crew.head()

In [ ]:
imdb_title_principals.head()

In [ ]:
tmdb_movies.head()

# The tn_movie_budgets set is merged with the imdb_title_basics set on the title column.  From this table we can select subsets based on date and genre.

In [ ]:
merge1 = tn_movie_budgets.rename(
    columns={'movie': 'primary_title'}).merge(imdb_title_basics, on= 'primary_title')
merge1.head()

### The unique occurances of genres is counted on indices.  Only genres that appear frequently are being used, so that rare instances of obscure genres do not significantly skew the data.

In [ ]:
indices = merge1.genres.value_counts()
indices.loc[indices>40] #genres that occur in movies over 40 times

# The budget gross ratio mean is calculated for the 15 most common genres

In [ ]:
import seaborn as sns
sns.set_style('darkgrid')

bgrmean = merge1.groupby('genres').budget_gross_ratio.mean()
indices = merge1.genres.value_counts()[:15].index

mean_of_top_genres = bgrmean[indices]
fig, ax = plt.subplots(figsize = (25,10))
ax.set_title('Mean budget to gross ratio of most frequent movie genres')
ax.set_ylabel('Mean of budget to gross ratio')
ax.set_xlabel('Genre')
ax.bar(indices, mean_of_top_genres);

# Median budget to gross ratio

In [ ]:
bgrmedian = merge1.groupby('genres').budget_gross_ratio.median()
indices = merge1.genres.value_counts()[:15].index

median_of_top_genres = bgrmedian[indices]
fig, ax = plt.subplots(figsize = (25,10))
ax.set_title('Median budget to gross ratio of most frequent movie genres')
ax.set_ylabel('Median of budget to gross ratio')
ax.set_xlabel('Genre')
ax.bar(indices, median_of_top_genres);

In [ ]:
dat = merge1.groupby('genres').worldwide_gross.mean()
indices = merge1.genres.value_counts()[:15].index
mean_of_top_genres = dat[indices]
fig, ax = plt.subplots(figsize = (25,10))
ax.set_title('Mean worldwide gross of most frequent movie genres')
ax.set_ylabel('Mean of worldwide gross')
ax.set_xlabel('Genre')
ax.bar(indices, mean_of_top_genres);

In [ ]:
imdb_title_basics.head()